In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from master_function import import_cot_data, data_preprocessing
from master_function import plot_train_test_values, calculate_directional_accuracy
from sklearn.metrics import mean_squared_error

# Calling the function and preprocessing the data
CAD = 'CANADIAN DOLLAR - CHICAGO MERCANTILE EXCHANGE'
NAS = 'NASDAQ-100 Consolidated - CHICAGO MERCANTILE EXCHANGE'
data = import_cot_data(2010, 2024, NAS)
data = np.array(data.iloc[:, -1], dtype = np.float64)

Selected: traders_in_financial_futures_fut
Downloaded single year data from: 2010
Stored the file FinFutYY.txt in the working directory.
Selected: traders_in_financial_futures_fut
Downloaded single year data from: 2011
Stored the file FinFutYY.txt in the working directory.
Selected: traders_in_financial_futures_fut
Downloaded single year data from: 2012
Stored the file FinFutYY.txt in the working directory.
Selected: traders_in_financial_futures_fut
Downloaded single year data from: 2013
Stored the file FinFutYY.txt in the working directory.
Selected: traders_in_financial_futures_fut
Downloaded single year data from: 2014
Stored the file FinFutYY.txt in the working directory.
Selected: traders_in_financial_futures_fut
Downloaded single year data from: 2015
Stored the file FinFutYY.txt in the working directory.
Selected: traders_in_financial_futures_fut
Downloaded single year data from: 2016
Stored the file FinFutYY.txt in the working directory.
Selected: traders_in_financial_futures_fu

/home/supercoder/Repos/Trading-Strategies/lstm/master_function.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Net_COT'] = (data['Pct_of_OI_Lev_Money_Long_All'] - \


In [7]:
data

array([-4.040e+01, -4.610e+01, -5.120e+01, -4.860e+01, -5.360e+01,
       -6.630e+01, -6.540e+01, -6.750e+01, -6.760e+01, -6.390e+01,
       -5.150e+01, -7.160e+01, -6.370e+01, -6.110e+01, -5.890e+01,
       -4.320e+01, -3.210e+01, -1.420e+01,  2.300e+00,  1.530e+01,
        1.590e+01, -4.000e-01, -2.280e+01, -1.920e+01, -3.550e+01,
       -3.530e+01, -4.990e+01, -5.890e+01, -5.930e+01, -7.160e+01,
       -7.780e+01, -8.220e+01, -8.680e+01, -8.360e+01, -8.060e+01,
       -7.140e+01, -5.570e+01, -6.570e+01, -7.070e+01, -7.300e+01,
       -6.190e+01, -4.090e+01, -3.050e+01, -3.740e+01, -2.910e+01,
       -3.530e+01, -4.670e+01, -5.020e+01, -4.360e+01, -5.050e+01,
       -6.910e+01, -6.940e+01, -7.590e+01, -7.580e+01, -7.820e+01,
       -8.140e+01, -7.190e+01, -6.320e+01, -5.460e+01, -4.830e+01,
       -3.900e+01, -2.740e+01, -1.750e+01,  3.200e+00,  1.340e+01,
        2.620e+01,  3.250e+01,  4.250e+01,  4.980e+01,  5.750e+01,
        7.100e+01,  7.110e+01,  6.680e+01,  6.470e+01,  4.970e

In [6]:
# Setting the hyperparameters
num_lags = 100
train_test_split = 0.80
neurons = 200
num_epochs = 200
batch_size = 4

# Creating the training and test sets
x_train, y_train, x_test, y_test = data_preprocessing(data, num_lags, train_test_split)

# Reshape the data to 3D for LSTM input
x_train = x_train.reshape((-1, num_lags, 1))
x_test = x_test.reshape((-1, num_lags, 1))

# Create the LSTM model
model = Sequential()

# Adding a first layer
model.add(LSTM(units = neurons, input_shape = (num_lags, 1)))

# Adding a second layer
model.add(Dense(neurons, activation = 'relu')) 

# Adding the output layer 
model.add(Dense(units = 1))

# Compiling the model
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

# Fitting the model
model.fit(x_train, y_train, epochs = num_epochs, batch_size = batch_size)

Epoch 1/200
82/82 [==============================] - 3s 21ms/step - loss: 777.4815
Epoch 2/200
82/82 [==============================] - 2s 21ms/step - loss: 169.2368
Epoch 3/200
82/82 [==============================] - 2s 21ms/step - loss: 157.4444
Epoch 4/200
82/82 [==============================] - 2s 20ms/step - loss: 154.0227
Epoch 5/200
82/82 [==============================] - 2s 21ms/step - loss: 139.5877
Epoch 6/200
82/82 [==============================] - 2s 22ms/step - loss: 137.4031
Epoch 7/200
82/82 [==============================] - 2s 21ms/step - loss: 133.0997
Epoch 8/200
82/82 [==============================] - 2s 21ms/step - loss: 133.1294
Epoch 9/200
82/82 [==============================] - 2s 20ms/step - loss: 140.3442
Epoch 10/200
82/82 [==============================] - 2s 21ms/step - loss: 137.8657
Epoch 11/200
82/82 [==============================] - 2s 21ms/step - loss: 125.7918
Epoch 12/200
55/82 [===================>..........] - ETA: 0s - loss: 125.9608

KeyboardInterrupt: 

In [ ]:
# Predicting in the training set for illustrative purposes
y_predicted_train = model.predict(x_train)

# Predicting in the test set
y_predicted = model.predict(x_test)

# Plotting
plot_train_test_values(100, 50, y_train, y_test, y_predicted)

# Performance evaluation
print('---')
print('Directional Accuracy Train = ', round(calculate_directional_accuracy(y_predicted_train, y_train), 2), '%')
print('Directional Accuracy Test = ', round(calculate_directional_accuracy(y_predicted, y_test), 2), '%')
print('RMSE Train = ', round(np.sqrt(mean_squared_error(y_predicted_train, y_train)), 10))
print('RMSE Test = ', round(np.sqrt(mean_squared_error(y_predicted, y_test)), 10))
print('Correlation In-Sample Predicted/Train = ', round(np.corrcoef(np.reshape(y_predicted_train, (-1)), y_train)[0][1], 3))
print('Correlation Out-of-Sample Predicted/Test = ', round(np.corrcoef(np.reshape(y_predicted, (-1)), np.reshape(y_test, (-1)))[0][1], 3))
print('---')